In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
from PIL import Image
import glob
import os
import re

%matplotlib inline

In [ ]:
filepath = "/data/asl_alphabet_{0}/"
cwd = os.getcwd()

In [ ]:
num_classes = 26
batch_size = 117
lr = 0.0001
dropout = 0.5
eps = 0.001

# Data Exploration and Preprocessing

In [ ]:
def get_imagepaths(type="train"):
    imagepaths = []

    for dir in glob.glob(cwd + filepath.format(type) + "*/"):
        for imagepath in glob.glob(dir + "*.jpg"):
            imagepaths.append(imagepath)
    imagepaths = np.array(imagepaths)
    np.random.shuffle(imagepaths)
    return imagepaths

In [ ]:
def build_data(imagepaths):
    data = []
    for p in imagepaths:
        label = re.search(r"\/([A-Z])\/", p).group(0)[1]
        data.append((p, label))

    return np.array(data)

In [ ]:
def train_val_split(imagepaths, val_amt):
    data = build_data(imagepaths)
    amt = data.shape[0]
    train = data[:int(amt*(1-val_amt))]
    val = data[int(amt*(1-val_amt)):]
    
    return train, val

In [ ]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def create_tfrecord(data, type="train"):
    tfr_dir = "/data/tfrecords/"
    if not os.path.exists(cwd + tfr_dir):
        os.makedirs(cwd + tfr_dir)
    tfr_filename = "{0}.tfrecords".format(type)
    if os.path.isfile(cwd + tfr_dir + tfr_filename):
        os.remove(cwd + tfr_dir + tfr_filename)
    writer = tf.python_io.TFRecordWriter(cwd + tfr_dir + tfr_filename)
    
    for i, p in enumerate(data):
        feature = {
            "train/image": _bytes_feature(p[0].encode()),
            "train/label": _bytes_feature(p[1].encode())
        }

        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
            
    print(tfr_filename + " Complete")
    writer.close()

In [ ]:
imagepaths = get_imagepaths("train")
train, val = train_val_split(imagepaths, 0.1)
print("Train data size: {0}".format(train.shape[0]))
print("Validation data size: {0}".format(val.shape[0]))
create_tfrecord(train, "train")
create_tfrecord(val, "validation")

In [ ]:

lr = tf.placeholder(tf.float64, name="learning_rate")
batch = tf.placeholder(tf.int64, name="batch_size")
train_filepath = tf.placeholder(tf.string, name="train_filepath")
val_filepath = tf.placeholder(tf.string, name="val_filepath")

In [ ]:
def setup_datasets():
    train_dataset = tf.data.TFRecordDataset(train_filepath)
    train_dataset = train_dataset.batch(batch)
    val_dataset = tf.data.TFRecordDataset(val_filepath)
    
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
    
    next_elem = iterator.get_next()
    train_iter = train_dataset.make_initializable_iterator()
    val_iter = val_dataset.make_initializable_iterator()

In [ ]:
def conv_layer(input, size_in, size_out, k_size, name):
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable("weights", shape=[k_size, k_size, size_in, size_out])
        biases = tf.get_variable("biases", shape=[size_out], initializer=tf.constant_initializer(0.0))
        conv = tf.nn.conv2d(input, weights, strides=[1,1,1,1], padding="SAME")
        conv = tf.bias_add(conv, biases)
        act = tf.nn.relu(conv, scope.name)
        
        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("activations", act)
        
    return act

def batch_norm_layer(input, is_training, eps, name):
    with tf.variable_scope(name) as scope:
        gamma = tf.get_variable("gamma", shape=[input.shape[0]], initializer=tf.constant_initializer(1.0))
        beta = tf.get_variable("beta", shape=[input.shape[0]], initializer=tf.constant_initializer(0.0))
        mean, var = tf.nn.moments(input, [0,1,2])
        ema = tf.nn.ExponentialMovingAverage(decay=0.99)
        
        def mean_var_with_update():
            ema_apply_op = ema.apply([mean, var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(mean), tf.identity(var)
            
        mean, var = tf.cond(is_training, mean_var_with_update, lambda: (ema.average(mean), ema.average(var)))
        bn = tf.nn.batch_normalization(input, mean, var, gamma, beta, eps)
        
        tf.summary.histogram("gamma", gamma)
        tf.summary.histogram("beta", beta)
        
    return bn
        
def dropout_layer(input, dropout):
    do = tf.nn.dropout(input, dropout)
    
    return do

def maxpool_layer(input, name):
    return tf.nn.max_pool(input, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME", name=name)

def fc_layer(input, size_in, size_out, name):
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable("weights", shape=[size_in, size_out])
        biases = tf.get_variable("biases", shape=[size_out])
        act = tf.matmul(reshape, weights) + biases
        
        tf.summary.histogram("weights", weights)
        tf.summary.histogram("biases", biases)
        tf.summary.histogram("activations", act)
        
    return act

In [ ]:
def model(X, dropout):
    tf.reset_default_graph()
    
    X = tf.placeholder(tf.float32, [None, 30, 30, 3], name="input")
    tf.summary.image("input", X, 10)
    Y = tf.placeholder(tf.int32, [None, num_classes], name="output")

    conv1 = conv_layer(X, 3, 64, 3, "conv1")
    conv2 = conv_layer(conv2, 64, 64, 3, "conv2")
    maxpool1 = maxpool_layer(conv2, "maxpool1")
    conv3 = conv_layer(maxpool1, 64, 64, 3, "conv3")
    conv4 = conv_layer(conv3, 64, 64, 3, "conv4")
    maxpool2 = maxpool_layer(conv4, "maxpool")
    flatten = tf.reshape(maxpool2, shape=[-1, tf.multiply(maxpool2.shape()[1:])])
    fc1 = fc_layer(flatten, flatten.shape()[1], 1024, "fc1")
    dropout1 = dropout_layer(fc1, dropout)
    fc2 = fc_layer(dropout1, dropout1.shape()[1], 1024, "fc2")
    dropout2 = dropout_layer(fc2, dropout)
    fc3 = fc_layer(dropout2, dropout2.shape()[1], 26, "fc3")

    return fc3

In [ ]:
def loss(logits, labels):
    with tf.name_scope("loss"):
        labels = tf.cast(labels, tf.int64)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels)
        cross_entropy_mean = tf.reduce_mean(cross_entropy, name="cross_entropy")

        tf.summary.scalar("cross entropy", cross_entropy_mean)
        
    return cross_entropy_mean

In [ ]:
def training(loss, learning_rate):
    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        train_op = optimizer.minimize(loss, global_step=global_step)

    return train_op

In [ ]:
def evaluation()

In [ ]:
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    tf.summary.scalar("accuracy", accuracy)